# QDNAseq

A normal human genome is comprised of 23 pairs of chromosomes. Each of these chromosomes is divided into 2 sections or arms: The small or “p” arm (from the French “petit”, small) and the long or “q” arm (because “q” comes after “p”), separated by a centromere. In cancer, chromosomal aberrations reshape the structure of the chromosomes, often resulting in the loss of whole arms or parts thereof.

![Chromosome](img/X2604-A-61.png)

The QDNAseq work of Daoud Sie was presented in the lectures. In this notebook, you will reproduce part of his results for reads from 6 different cancer cell lines, using the accompanying R package. Note that although this uses a programming language you might not be familiar with, you do not necessarily have to learn R for this assignment, as *most* of the syntax is 1) given to you and/or 2) self-explanatory.

If you want more information about what specific functions do, refer to the [package documentation](https://bioconductor.org/packages/release/bioc/manuals/QDNAseq/man/QDNAseq.pdf) or use **`help(`*`<function name>`*`)`**

To answer the questions on Canvas, you should know *what* each function does, and *why* this is necessary or useful. You may find it helpful to look at the questions while working through this notebook.

<div class="alert alert-block alert-info"><b>Note:</b> As you've already seen how to turn FASTQ reads into a BAM file in the previous notebook, we saved you a lot of compute time by mapping the reads for the cell lines in this exercise against the entire human genome (hg19) <i>for</i> you. Thus, you do not have to align anything yourself. You can find each cell line's mapped reads in the <b>~/bam</b> folder.</div>

In [ ]:
# Load the R package
library(QDNAseq)

In [ ]:
# Download bin annotations

bins = getBinAnnotations(binSize=30)

In [ ]:
# Get read counts per bin

bamFiles = Sys.glob("/local/data/bsb_asa/student/bam/SW480*.bam")
# Uncomment the line below to perform analysis for ALL cell lines at once!
# bamFiles = Sys.glob("/local/data/bsb_asa/student/bam/*.bam") 
readCounts = binReadCounts(bins, bamfiles = bamFiles)

In [ ]:
# Plot the counts per bin

plot(readCounts, logTransform=FALSE, ylim=c(-50, 200))
highlightFilters(readCounts, logTransform=FALSE, residual=TRUE, blacklist=TRUE)
# Tip: Investigate what this highlightFilters() function does!

In [ ]:
# Apply filters and draw isobarPlot(s)

readCountsFiltered = applyFilters(readCounts, residual=TRUE, blacklist=TRUE)
isobarPlot(readCountsFiltered)

In [ ]:
# Correct each bin's count for the relationship between GC content and mappability
readCountsFiltered = estimateCorrection(readCountsFiltered)

# If relevant, visualize average reads per bin vs. the variance for each cell line
if (length(bamFiles) > 1) {noisePlot(readCountsFiltered)}

In [ ]:
# Magic or madness? Make sure you understand these functions!

copyNumbers = correctBins(readCountsFiltered)
copyNumbersNormalized = normalizeBins(copyNumbers)
copyNumbersSmooth = smoothOutlierBins(copyNumbersNormalized)
plot(copyNumbersSmooth)

In [ ]:
# Do segmentation on the smoothed read counts, normalize, and plot the result

copyNumbersSegmented = segmentBins(copyNumbersSmooth, transformFun="log2")
copyNumbersSegmented = normalizeSegmentedBins(copyNumbersSegmented)
plot(copyNumbersSegmented)

<div class="alert alert-block alert-info"> Here is a reference frequency plot of copy number gains and losses from colorectal cancers made from 105 patient samples. Comparing the segmentation profiles of your analyzed cell line(s) may help you answer some of the questions, as well as understand how the cell lines are similar or different from "typical" cases!</div>

![A frequency plot of colorectal cancers from 105 samples.](img/profile.png)

In [ ]:
# Some model-based statistics on the assigned segmentation...
# NOT recommended for this assignment, because figures are hard to read!
# Only check if you are curious!

copyNumbersCalled = callBins(copyNumbersSegmented)
plot(copyNumbersCalled)